In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/biaspairclassification/text-and-id.txt
/kaggle/input/biaspairclassification/pairs-label-training (1).txt


In [3]:
import numpy as np
import pandas as pd
text_df = pd.read_csv('/kaggle/input/biaspairclassification/text-and-id.txt',sep='\n',names=['col'])
text_df

,col
0,"57 Every person has a right to profess, practi..."
1,"1603 That is , the pigment particles mix with ..."
2,755 The Prime Minister of India is the leader ...
3,1636 The water you drink now-a-days is not saf...
4,"828 I , along with my mother , brothers and si..."
...,...
1995,222 But her agriculture is underdeveloped and ...
1996,926 Teacher : A teacher helps his students wit...
1997,490 She should take steps to fulfill demands o...
1998,1293 Look carefully at how the tendon joins th...


In [40]:
pair_df = pd.read_csv('/kaggle/input/biaspairclassification/pairs-label-training (1).txt',names=['p1','p2','label'])
pair_df

,p1,p2,label
0,98,651,0
1,696,1870,1
2,1073,1620,0
3,167,633,0
4,4,640,0
...,...,...,...
155946,546,1512,1
155947,720,1902,1
155948,1112,1591,0
155949,396,1472,1


In [4]:
text_df[['id', 'text']] = text_df['col'].str.split(n=1, expand=True)
text_df.id = text_df.id.astype('int')
text_df.drop('col',axis=1,inplace=True)

In [5]:
text_df = text_df.set_index('id').sort_index()
text_df

,text
id,
1,Besides these forests naturally developed fore...
2,Your mother cleans the rice by separating husk...
3,All free gifts of nature and all things which ...
4,Humanism - the concept that man is the measure...
5,Various designs in clothing are found dependin...
...,...
1996,Bring both the ends of threads together and pu...
1997,What is the duration of the longest day in Dec...
1998,PotassiumProtein synthesisEnables the opening ...


In [6]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 16.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 29.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.9
    Uninstalling blis-0.7.9:
      Successfully uninstalled blis-0.7.9
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing inst

In [7]:
!pip install text-hammer
import text_hammer as th
def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    return(df)  

In [8]:
text_df = text_preprocessing(text_df, 'text')


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [9]:
feel_arr = text_df.text
from nltk.tokenize import word_tokenize,sent_tokenize

feel_arr = [word_tokenize(sent) for sent in feel_arr]
print(feel_arr[0])

['besides', 'these', 'forest', 'naturally', 'develop', 'forest', 'there', 'are', 'man', 'make', 'forest', 'too', 'which', 'are', 'develop', 'artificially', 'over', 'a', 'vast', 'area', 'of', 'land']


In [10]:
def padd(arr):
    for i in range(100-len(arr)):
        arr.append('<pad>')
    return arr[:100]
   
for i in range(len(feel_arr)):
    feel_arr[i]=padd(feel_arr[i])

In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-11-01 20:17:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-01 20:17:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-01 20:17:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [12]:
!pwd

/kaggle/working


In [52]:
vocab_f = r'/kaggle/working/glove.6B.300d.txt'
 
# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 300d vector.
embeddings_index = {}
with open(vocab_f, encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of items- in which
        # the first element is the word to be embedded, and from second
        # to the end of line contains the 300d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Now, each word of the dataset should be embedded in 300d vector with
# the help of the dictionary form above.

# Embedding each word of the feel_arr
embedded_feel_arr = []
for each_sentence in feel_arr:
    embedded_feel_arr.append([])
    for word in each_sentence:
        if word.lower() in embeddings_index:
            embedded_feel_arr[-1].append(embeddings_index[word.lower()])
        else:
                # if the word to be embedded is '<padd>' append 0 3(hundred) times
            embedded_feel_arr[-1].append([0]*300)

In [53]:
X_train = np.array(embedded_feel_arr)

In [54]:
X_train.shape

(2000, 100, 300)

In [55]:
X_train

array([[[-0.056492  ,  0.085955  ,  0.18274   , ..., -0.11703   ,
         -0.11546   ,  0.19175   ],
        [-0.19491   ,  0.14845   , -0.052926  , ..., -0.31865001,
         -0.066716  , -0.17591999],
        [-0.45533001,  0.14614999, -0.028433  , ...,  0.13561   ,
          0.16079   ,  0.18308   ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.36778   , -0.1084    , -0.10166   , ...,  0.01278   ,
         -0.31737   ,  0.13973001],
        [-0.34762999,  0.0055901 , -0.42938   , ..., -0.13698   ,
         -0.19644   ,  0.84759003],
        [ 0.27832001, -0.044424  , -0.55756003, ..., -0.33991   ,
         -0.19088   ,  0.41248   ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [56]:
from sklearn.cluster import KMeans, MiniBatchKMeans
# model = KMeans(n_clusters=4, init='k-means++', max_iter=100)
# model.fit(X_train)

whee


In [57]:
'hello'

'hello'

In [58]:
# from nltk.cluster import KMeansClusterer
# import nltk
# NUM_CLUSTERS=4
# kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
# assigned_clusters = kclusterer.cluster(X_train, assign_clusters=True)
# print(assigned_clusters)

In [59]:
X_train_reshaped = X_train.reshape(2000,5000)

ValueError: cannot reshape array of size 60000000 into shape (2000,5000)

In [60]:
from sklearn.cluster import KMeans, MiniBatchKMeans
model = KMeans(n_clusters=4, init='k-means++', max_iter=100)
model.fit(X_train_reshaped)

KMeans(max_iter=100, n_clusters=4)

In [61]:
text_df["bias"] = model.predict(X_train_reshaped)
text_df["bias"]=text_df["bias"].replace({0:0,1:1,2:1,3:1})
# text_df = text_df.sort_values(by=["id"]).reset_index().drop("index",axis=1)
# pred_bias = []

# for i in range(len(df)):
#     pred_bias.append((text_df["bias"][df["p1"][i]-1] ^ text_df["bias"][df["p2"][i]-1]))
# print(classification_report(df["bias"],pred_bias))
# print(confusion_matrix(df["bias"],pred_bias))

In [62]:
text_df

,text,bias
id,,
1,besides these forest naturally develop forest ...,1
2,your mother clean the rice by separate husk mu...,1
3,all free gift of nature and all thing which ma...,1
4,humanism the concept that man is the measure o...,1
5,various design in clothing are find depend on ...,1
...,...,...
1996,bring both the end of thread together and put ...,1
1997,what is the duration of the long day in december,1
1998,potassiumprotein synthesisenable the opening a...,0


In [63]:
pair_df

,p1,p2,label
0,98,651,0
1,696,1870,1
2,1073,1620,0
3,167,633,0
4,4,640,0
...,...,...,...
155946,546,1512,1
155947,720,1902,1
155948,1112,1591,0
155949,396,1472,1


In [64]:
pred_label = []
for i in range(pair_df.shape[0]):
    pred_label.append(np.logical_xor(text_df.bias[pair_df.p1[i]],text_df.bias[pair_df.p2[i]]).astype(int))
#     break;

In [65]:
from sklearn.metrics import classification_report
print(classification_report(pair_df.label.values,pred_label))

              precision    recall  f1-score   support

           0       0.51      0.78      0.62     80400
           1       0.47      0.21      0.29     75551

    accuracy                           0.50    155951
   macro avg       0.49      0.49      0.45    155951
weighted avg       0.49      0.50      0.46    155951

